## GPT API calls for Direction 2 Constraint generation

In [14]:
system_prompt = """You are an English writing expert and you need to set hard essay prompts for the genre of Realistic Fiction. Your aim is to generate extremely hard constraints that make the essay writing very challenging. 

An essay prompt is defined as a main Instruction + Constraints. You will be given an Instruction as an input and you should generate more complex constraints that can be added to the Instruction. 

Constraints can be defined in the following categories - 
Content - Egs: Climate change impacts, .
Linguistic - Egs: Victorian English, Pirate slang.
Style - Egs: Respectful Tone, ELI5
Format - Egs: List 3 bullet points, paragraphs.
Number - Egs: 500 word essay, three arguments for electric vehicles.

The definitions are just for your reference.

Give me a numbered list of EXACTLY {} CONSTRAINTS.

The constraints should be complex and creative but as the genre is Realistic Fiction, be true to the genre. The constraints should be clear and atomic, that is, if a constraint can be decomposed into multiple sub constraints, list all of them separately. Although challenging, it should be possible to write a coherent story with a flow using your constraints.

IMPORTANT: Your constraints should be UNIQUE. DO NOT GIVE CONTRADICTORY CONSTRAINTS. It should NOT require any specific domain knowledge to understand the constraints. DO NOT INCLUDE COMPLICATED OR POETRY RELATED VOCABULARY in your responses. DO NOT START YOUR LINE WITH THE CONSTRAINT CATEGORY (Lisguistic:, Format:, etc), JUST GIVE ONLY THE CONSTRAINT.

"""

prompt_examples = """Here are some example responses -

Input - Write a story about the incident of two friend's day out gone wrong.
Output - 
1. The story must be written in the form of a dialogue between the two friends.
2. The story must include a flashback to a childhood memory of one of the friends.
3. One of the friends must have a secret that is revealed during the day out.
4. The story must contain at least three unexpected plot twists.
5. The friends must communicate only through handwritten notes for a part of the story.
6. The story must include a mysterious stranger who impacts the day out.
7. One of the friends must have a phobia that becomes a central theme in the story.
8. The setting of the story must change at least three times.
9. The climax of the story must involve a natural disaster.
10. The story must end with a cliffhanger that leaves the readers guessing about the fate of the friends.

Input - Write a story that follows the journey of a professional woman of color named Rachel Michelle.
Output - 
1. Rachel is single, has two kids she is supporting through high school.
2. Rachel is considering leaving her corporate job to become an entrepreneur. 
3. The story should detail the steps she takes to create a life plan and make the leap.
4. Highlight her success as an entrepreneur with just five employees.
5. The story should be written in a motivational and engaging style.
6. Incorporate practical and mental wellness strategies that helped her achieve her goals.
7. Explore the complex relationship between her and her growing kids.
8. The story should be a lesson in financial freedom.
9. The story should cater to a young audience.
10. Write the story in 1000 words or less.

Input - Imagine a world where life expectancy is fixed and same for everyone at birth, yet people can engage in life expectancy "donation" - meaning that someone can extend your life three years longer at the expense of theirs. 
Output -
1. Include themes of manipulation and forced donation.
2. Follow a group of teenage kids who live in this world.
3. Maintain a dark, gritty theme and atmosphere.
4. Write the story in 4 parts - each part being less than 300 words.
5. Include a dog's perspective. 
6. Introduce rules and policies that might seem extreme but are normal in this world.
7. Explore how fashion is different in this world.
8. Incorporate humorous characters to offset the dark setting of this world.
9. Show how unusual morning routines can get.
10. End the story on a cliffhanger.

"""
user_input = """
Input - {}
Output -
"""

Setup OpenAI Model

In [15]:
from openai import OpenAI
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=api_key)

def chat(instruction, model="gpt-3.5-turbo", system_prompt=system_prompt, n=20, log=False):
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content":system_prompt.format(n)},
            {"role": "user", "content":user_input.format(instruction)},
        ]
    )

    log_usage(tokens=response.usage.total_tokens)

    if log:
        print("Total tokens used: ", response.usage.total_tokens)

    return response


def log_usage(tokens):
    # Get the current date and time
    current_time = datetime.now().strftime("%m-%d-%Y %H:%M:%S")

    # Write the date-time and tokens used to the file
    with open("api_usage.txt", "a") as file:
        file.write(f"{current_time} : {tokens}\n")


def total_usage():
    total_tokens = 0
    with open("api_usage.txt", "r") as file:
        for line in file:
            # Split the line into date-time and tokens
            parts = line.split(" : ")
            if len(parts) == 2:
                _, tokens_str = parts
                tokens = int(tokens_str)
                total_tokens += tokens
    
    cost = (total_tokens*0.0015)/1000
    print("Total tokens used so far: ", total_tokens)
    print(f"Total cost so far: {cost}$")
    return



In [16]:
import openpyxl
import pandas as pd

def get_instructions():

    # Read the Excel file into a DataFrame
    df = pd.read_excel("Data/input_instructions.xlsx")

    instructions = [list(row.values()) for row in df.to_dict(orient='records')]
    
    return instructions

In [17]:
def generate_constraints():
    n = 20
    instructions = get_instructions()
    sheet_name = "chatgpt-3.5-turbo"

    df = pd.read_excel('Data/constraints.xlsx', sheet_name=sheet_name)

    dfs_to_concat = []

    for instruction in instructions:
        
        response = chat(instruction=instruction[0], n=n, log=True)

        new_row = {
            'Instruction': instruction[0],
            'Category': instruction[1],
            'Constraints': response.choices[0].message.content
        }
        new_df = pd.DataFrame([new_row])

        dfs_to_concat.append(new_df)
    
    # Concatenate all DataFrames in the list
    df = pd.concat([df] + dfs_to_concat, ignore_index=True)

    df.to_excel('Data/constraints.xlsx', index=False, sheet_name=sheet_name)

    

In [18]:
generate_constraints()

Total tokens used:  737
Total tokens used:  710
Total tokens used:  800
Total tokens used:  785
Total tokens used:  778
Total tokens used:  810
Total tokens used:  882
Total tokens used:  930
Total tokens used:  758
Total tokens used:  813


In [20]:
total_usage()

Total tokens used so far:  27810
Total cost so far: 0.041715$


In [153]:
instructions = get_instructions()
n = 20
for instruction in instructions:
    # print(instruction[0])
    messages=[
        {"role": "user", "content":user_input.format(instruction[0])},
    ]
    print(messages)

[{'role': 'user', 'content': '\nInput - Plan for a three day road trip for May long weekend from Westfield, New Jersey\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a story about incident of two friend’s day out going wrong.\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a short paragraph about why you should not have both a pet cat and a pet bird.\t\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a passage for ideas for the most amazing welcome party in high school.\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Plan a weekend long vacation to Madrid, and mention multiple specific places you would visit and how you would travel in the city\t\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a story about being stuck in a locked gym storage room with the person you arguably hate the most.\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a story about returning something a random stranger dropped on the train, late